In [1]:
from src.arguments import ModelArguments, DataArguments
from src.model.model import MMEBModel
from src.model.processor import load_processor, QWEN2_VL, VLM_IMAGE_TOKENS, \
    Qwen2_VL_process_fn, LLAVA_QWEN2, FastVLM_process_fn
from src.utils import batch_to_device
from PIL import Image
import numpy as np
import torch
import math
%matplotlib inline
import matplotlib.pyplot as plt
import torch.nn.functional as F

from transformers.image_transforms import (
    convert_to_rgb,
    resize,
)

/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/workspace/VLM_Embed/vlm/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/workspace/VLM_Embed/src/model/vlm_backbone/internvideo2/modeling_internvideo2.py:541: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
[2026-01-02 12:56:08,608] DEBUG [matplotlib:342] matplotlib data path: /workspace/VLM_Embed/vlm/lib/python3.12/site-packages/matplotlib/mpl-data
[2026-01-02 12:56:08,614] DEBUG [matplotlib:342] CONFIG

FusedMLP of flash_attn is not installed!!!
DropoutAddRMSNorm of flash_attn is not installed!!!
flash_attn_interface or bert_padding of flash_attn is not installed!!!


[2026-01-02 12:56:08,646] DEBUG [matplotlib:342] CACHEDIR=/root/.cache/matplotlib
[2026-01-02 12:56:08,648] DEBUG [matplotlib.font_manager:1635] Using fontManager instance from /root/.cache/matplotlib/fontlist-v390.json
[2026-01-02 12:56:08,833] DEBUG [matplotlib.pyplot:496] Loaded backend module://matplotlib_inline.backend_inline version unknown.
[2026-01-02 12:56:08,835] DEBUG [matplotlib.pyplot:496] Loaded backend inline version unknown.


In [2]:
# model_args = ModelArguments(
#     model_name='Qwen/Qwen2-VL-2B',
#     checkpoint_path='TIGER-Lab/VLM2Vec-Qwen2VL-2B',
#     pooling='last',
#     normalize=True,
#     model_backbone='qwen2_vl',
#     lora=True
# )
model_args = ModelArguments(
    model_name='apple/FastVLM-0.5B',
    pooling='last',
    normalize=True,
    model_backbone=LLAVA_QWEN2,
)
data_args = DataArguments()

processor = load_processor(model_args, None)
model = MMEBModel.build(model_args)
model = model.to('cuda', dtype=torch.bfloat16)
model.eval()

[2026-01-02 12:56:11,574] INFO [src.utils:21] Loading processor from: apple/FastVLM-0.5B


Processor load here for LLAVA-QWEN2


[2026-01-02 12:56:11,604] DEBUG [urllib3.connectionpool:1049] Starting new HTTPS connection (1): huggingface.co:443
[2026-01-02 12:56:13,105] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/tokenizer_config.json HTTP/1.1" 307 0
[2026-01-02 12:56:13,327] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/tokenizer_config.json HTTP/1.1" 200 0
[2026-01-02 12:56:13,749] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "GET /api/models/apple/FastVLM-0.5B/tree/main/additional_chat_templates?recursive=False&expand=False HTTP/1.1" 404 64
[2026-01-02 12:56:14,660] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/config.json HTTP/1.1" 307 0
[2026-01-02 12:56:14,875] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/1

Detected model type: llava_qwen2
Determined model backbone: llava_qwen2


[2026-01-02 13:02:57,465] DEBUG [urllib3.connectionpool:289] Resetting dropped connection: huggingface.co
[2026-01-02 13:02:59,395] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/generation_config.json HTTP/1.1" 307 0
[2026-01-02 13:02:59,644] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /api/resolve-cache/models/apple/FastVLM-0.5B/16375720c2d673fa583e57e9876afde27549c7d0/generation_config.json HTTP/1.1" 200 0
[2026-01-02 13:03:00,055] DEBUG [urllib3.connectionpool:544] https://huggingface.co:443 "HEAD /apple/FastVLM-0.5B/resolve/main/custom_generate/generate.py HTTP/1.1" 404 0


MMEBModel(
  (encoder): LlavaQwen2ForCausalLM(
    (model): LlavaQwen2Model(
      (embed_tokens): Embedding(151936, 896)
      (layers): ModuleList(
        (0-23): 24 x Qwen2DecoderLayer(
          (self_attn): Qwen2Attention(
            (q_proj): Linear(in_features=896, out_features=896, bias=True)
            (k_proj): Linear(in_features=896, out_features=128, bias=True)
            (v_proj): Linear(in_features=896, out_features=128, bias=True)
            (o_proj): Linear(in_features=896, out_features=896, bias=False)
          )
          (mlp): Qwen2MLP(
            (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
            (up_proj): Linear(in_features=896, out_features=4864, bias=False)
            (down_proj): Linear(in_features=4864, out_features=896, bias=False)
            (act_fn): SiLU()
          )
          (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
          (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        )
      )
   

In [3]:
processor_inputs = {
    "text": [f'{VLM_IMAGE_TOKENS[LLAVA_QWEN2]} Represent the given image with the following question: What is in the image',
          f'{VLM_IMAGE_TOKENS[LLAVA_QWEN2]} Represent the given image with the following question: What is in the image'],
    "images": [Image.open('example.jpg').resize((500, 1000)),
            Image.open('example.jpg')],
}

inputs = FastVLM_process_fn(
    processor_inputs,
    processor, 
    # square_padding=True
    )
inputs = batch_to_device(inputs, "cuda")
inputs['images'][0].shape



torch.Size([3, 2048, 1024])

In [4]:
model.encode_input(inputs)

KeyboardInterrupt: 

In [ ]:
x = torch.randn(1, 3, 768, 768).to('cuda', dtype=torch.bfloat16)
with torch.no_grad():
    y = model.encoder.get_vision_tower()(x)
y.shape

RuntimeError: Given input size: (3072x12x12). Calculated output size: (3072x0x0). Output size is too small